# Acrobot

## A2C Agent 

In [ ]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [1]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent
if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    beta = 0.5

agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    first_state = env.reset()
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        avg_episode_reward = episode_reward/env.action_space.n
    
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2

    policy_loss = agent.compute_policy_loss_kl(state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -500.0
Episode 1: -382.6666666666667
Episode 2: -289.6666666666667
Episode 3: -230.33333333333334
Episode 4: -202.33333333333334
Episode 5: -174.66666666666666
Episode 6: -165.66666666666666
Episode 7: -178.66666666666666
Episode 8: -173.66666666666666
Episode 9: -140.33333333333334
Episode 10: -209.33333333333334
Episode 11: -145.66666666666666
Episode 12: -142.66666666666666
Episode 13: -210.33333333333334
Episode 14: -165.0
Episode 15: -186.66666666666666
Episode 16: -145.0
Episode 17: -133.0
Episode 18: -145.0
Episode 19: -164.33333333333334
Episode 20: -141.33333333333334
Episode 21: -155.66666666666666
Episode 22: -167.0
Episode 23: -135.0
Episode 24: -146.33333333333334
Episode 25: -159.0
Episode 26: -143.33333333333334
Episode 27: -121.33333333333333
Episode 28: -145.0
Episode 29: -150.33333333333334
Episode 30: -214.33333333333334
Episode 31: -165.33333333333334
Episode 32: -206.0
Episode 33: -200.66666666666666
Episode 34: -141.33333333333334
Episode 35: -163.66666

Episode 295: -107.66666666666667
Episode 296: -127.0
Episode 297: -114.66666666666667
Episode 298: -124.66666666666667
Episode 299: -99.33333333333333


In [ ]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [1]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    
agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    first_state = env.reset()
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        avg_episode_reward = episode_reward/env.action_space.n
    
    # add randomness for better exploration in the beginning
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta = beta + np.random.random()*0.05
    # print(beta)
    policy_loss = agent.compute_policy_loss_wass(state, state_adv, beta)

    
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -500.0
Episode 9: -500.0
Episode 10: -500.0
Episode 11: -500.0
Episode 12: -500.0
Episode 13: -500.0
Episode 14: -500.0
Episode 15: -500.0
Episode 16: -500.0
Episode 17: -500.0
Episode 18: -500.0
Episode 19: -500.0
Episode 20: -500.0
Episode 21: -500.0
Episode 22: -500.0
Episode 23: -500.0
Episode 24: -500.0
Episode 25: -500.0
Episode 26: -500.0
Episode 27: -500.0
Episode 28: -500.0
Episode 29: -500.0
Episode 30: -500.0
Episode 31: -500.0
Episode 32: -500.0
Episode 33: -500.0
Episode 34: -500.0
Episode 35: -500.0
Episode 36: -500.0
Episode 37: -500.0
Episode 38: -500.0
Episode 39: -500.0
Episode 40: -500.0
Episode 41: -500.0
Episode 42: -500.0
Episode 43: -500.0
Episode 44: -500.0
Episode 45: -500.0
Episode 46: -500.0
Episode 47: -500.0
Episode 48: -500.0
Episode 49: -500.0
Episode 50: -406.3333333333333
Episode 51: -350.333333333333

In [2]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')